# this is a test

In [1]:
from urllib.request import urlopen
import re
import json
import pandas as pd

In [ ]:
class RevisionViewer():
    def __init__(self, lang='en', article='Earth', pages_back_cap=None):
      self.lang = lang
      self.article=article
      self.url_core = f'https://api.wikimedia.org/core/v1/wikipedia/{lang}'
      self.revisions = []
      self.pages_back_cap = pages_back_cap


    def get_page_history_url(self):
        return self.url_core + f'/page/{self.article}/history'
    
    def get_revision_url(self, fromId: str, toId: str):
        return self.url_core + f'/revision/{fromId}/compare/{toId}'

    def load_revision(self, url):
        response = urlopen(url)
        return json.load(response)

    def load_revisions(self):
        url_next = self.get_page_history_url()
        index = 1

        while url_next and index <= self.pages_back_cap:
            print(f"Getting revisions: page {index}")
            revision_dict = self.load_revision(url_next)
            print(revision_dict)
            self.revisions.extend(revision_dict['revisions'])
            url_next = revision_dict.get('older', False)
            index = index + 1

        return self.revisions
        
    def get_df(self):
        self.load_revisions()
        df = pd.DataFrame(self.revisions)
        return df

    def get_revision_details(self, rev_id: str):
        prev_id = 
        url = rev_id



In [ ]:
viewer = RevisionViewer('pl', 'Polska', 3)
our_df = viewer.get_df()

Getting revisions: page 1
{'revisions': [{'id': 75146577, 'timestamp': '2024-11-05T21:15:15Z', 'minor': False, 'size': 324847, 'comment': 'Zmieniona ilustracja (1) - lepsza jakość portretu wg Matejki', 'user': {'id': 1060959, 'name': 'Redaktor GLAM'}, 'delta': 19}, {'id': 75141926, 'timestamp': '2024-11-05T12:51:28Z', 'minor': False, 'size': 324828, 'comment': '/* Zabór austriacki */ stylistyka', 'user': {'id': 971727, 'name': 'Dzikiłoś'}, 'delta': 0}, {'id': 75141587, 'timestamp': '2024-11-05T12:15:04Z', 'minor': False, 'size': 324828, 'comment': '/* Prehistoria ziem polskich */ jęz.', 'user': {'id': 971727, 'name': 'Dzikiłoś'}, 'delta': 1}, {'id': 75043624, 'timestamp': '2024-10-24T20:17:15Z', 'minor': False, 'size': 324827, 'comment': 'styl', 'user': {'id': 511, 'name': 'Stok'}, 'delta': -41}, {'id': 75043418, 'timestamp': '2024-10-24T19:41:45Z', 'minor': False, 'size': 324868, 'comment': 'Aktualizacja najbardziej i najmniej zagęszczonego miasta', 'user': {'id': 1267364, 'name': 'Bi

In [10]:
our_df

,id,timestamp,minor,size,comment,user,delta
0,75146577,2024-11-05T21:15:15Z,False,324847,Zmieniona ilustracja (1) - lepsza jakość portr...,"{'id': 1060959, 'name': 'Redaktor GLAM'}",19
1,75141926,2024-11-05T12:51:28Z,False,324828,/* Zabór austriacki */ stylistyka,"{'id': 971727, 'name': 'Dzikiłoś'}",0
2,75141587,2024-11-05T12:15:04Z,False,324828,/* Prehistoria ziem polskich */ jęz.,"{'id': 971727, 'name': 'Dzikiłoś'}",1
3,75043624,2024-10-24T20:17:15Z,False,324827,styl,"{'id': 511, 'name': 'Stok'}",-41
4,75043418,2024-10-24T19:41:45Z,False,324868,Aktualizacja najbardziej i najmniej zagęszczon...,"{'id': 1267364, 'name': 'Biała Łza'}",-15
5,75043386,2024-10-24T19:36:14Z,False,324883,Aktualizacja największego miasta,"{'id': 1267364, 'name': 'Biała Łza'}",-122
6,75043356,2024-10-24T19:31:32Z,False,325005,Aktualizacja najludniejszego i najmniej ludneg...,"{'id': 1267364, 'name': 'Biała Łza'}",-137
7,75043288,2024-10-24T19:23:28Z,False,325142,Poprawa błędnej i nieaktualnej informacji o li...,"{'id': 1267364, 'name': 'Biała Łza'}",-25
8,75034670,2024-10-23T22:32:23Z,False,325167,/* Nowożytność */ ilustracje,"{'id': 274492, 'name': 'Tarnoob'}",-4
9,75034666,2024-10-23T22:31:20Z,False,325171,/* Średniowiecze */ ilustracje,"{'id': 274492, 'name': 'Tarnoob'}",-11


In [38]:
response = urlopen(url)

In [39]:
json.load(response)

{'revisions': [{'id': 75146577,
   'timestamp': '2024-11-05T21:15:15Z',
   'minor': False,
   'size': 324847,
   'comment': 'Zmieniona ilustracja (1) - lepsza jakość portretu wg Matejki',
   'user': {'id': 1060959, 'name': 'Redaktor GLAM'},
   'delta': 19},
  {'id': 75141926,
   'timestamp': '2024-11-05T12:51:28Z',
   'minor': False,
   'size': 324828,
   'comment': '/* Zabór austriacki */ stylistyka',
   'user': {'id': 971727, 'name': 'Dzikiłoś'},
   'delta': 0},
  {'id': 75141587,
   'timestamp': '2024-11-05T12:15:04Z',
   'minor': False,
   'size': 324828,
   'comment': '/* Prehistoria ziem polskich */ jęz.',
   'user': {'id': 971727, 'name': 'Dzikiłoś'},
   'delta': 1},
  {'id': 75043624,
   'timestamp': '2024-10-24T20:17:15Z',
   'minor': False,
   'size': 324827,
   'comment': 'styl',
   'user': {'id': 511, 'name': 'Stok'},
   'delta': -41},
  {'id': 75043418,
   'timestamp': '2024-10-24T19:41:45Z',
   'minor': False,
   'size': 324868,
   'comment': 'Aktualizacja najbardziej i n